# Add Service Alerts Functions

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp_data_analysis.sql import query_sql
import calitp_data_analysis.magics
import branca

import utils

from siuba import *
import pandas as pd

from shared_utils import gtfs_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Quick example of finding cancelled service in service alerts

In [2]:
alert_query = '''SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
WHERE date BETWEEN '2022-05-11' AND '2022-05-12' AND calitp_itp_id in (290, 300)'''

In [3]:
alert_twodate = query_sql(alert_query)

In [4]:
#finding the unique service messages
alert_twodate.effect.unique()

array([None, 'UNKNOWN_EFFECT', 'REDUCED_SERVICE', 'MODIFIED_SERVICE',
       'ACCESSIBILITY_ISSUE', 'NO_SERVICE', 'STOP_MOVED', 'NO_EFFECT',
       'OTHER_EFFECT', 'DETOUR', 'ADDITIONAL_SERVICE'], dtype=object)

In [5]:
#Checking how many calitp_urls there are
alert_twodate>>group_by(_.calitp_itp_id)>>summarize(n = _.calitp_url_number.unique())

,calitp_itp_id,n
0,290,1
1,300,0


In [6]:
#filter for no service
alert_twodate>>filter(_.effect=="NO_SERVICE")

,calitp_itp_id,calitp_url_number,original_file_path,date,id,active_periods,informed_entities,cause,effect,url_translations,header_text_translations,description_text_translations,tts_header_text_translations,tts_description_text_translations,severity_level,key
6459,290,1,gtfs-data/rt/2022-05-12T21:43:40/290/1/gtfs_rt...,2022-05-12,FS_9f5f4e62-c99f-45c1-9ffd-3c19b61d1e41,"[{'start': 1652393700, 'end': 1652404500}]","[{'agencyId': 'FS', 'routeId': '2', 'routeType...",OTHER_CAUSE,NO_SERVICE,[],[],[],[],[],None,18525cd36e0f75e935f9fb0d6fa9f52d
6461,290,1,gtfs-data/rt/2022-05-12T21:44:00/290/1/gtfs_rt...,2022-05-12,FS_9f5f4e62-c99f-45c1-9ffd-3c19b61d1e41,"[{'start': 1652393700, 'end': 1652404500}]","[{'agencyId': 'FS', 'routeId': '2', 'routeType...",OTHER_CAUSE,NO_SERVICE,[],[],[],[],[],None,f1229356744e5fe5a595ad1b1956876c
6469,290,1,gtfs-data/rt/2022-05-12T21:44:20/290/1/gtfs_rt...,2022-05-12,FS_9f5f4e62-c99f-45c1-9ffd-3c19b61d1e41,"[{'start': 1652393700, 'end': 1652404500}]","[{'agencyId': 'FS', 'routeId': '2', 'routeType...",OTHER_CAUSE,NO_SERVICE,[],[],[],[],[],None,98c58b892afd7e62364d368cbbd85457
6483,290,1,gtfs-data/rt/2022-05-12T21:44:40/290/1/gtfs_rt...,2022-05-12,FS_9f5f4e62-c99f-45c1-9ffd-3c19b61d1e41,"[{'start': 1652393700, 'end': 1652404500}]","[{'agencyId': 'FS', 'routeId': '2', 'routeType...",OTHER_CAUSE,NO_SERVICE,[],[],[],[],[],None,b96fbc7b5606c13cf30a2c4fbb015c55
6485,290,1,gtfs-data/rt/2022-05-12T21:45:00/290/1/gtfs_rt...,2022-05-12,FS_9f5f4e62-c99f-45c1-9ffd-3c19b61d1e41,"[{'start': 1652393700, 'end': 1652404500}]","[{'agencyId': 'FS', 'routeId': '2', 'routeType...",OTHER_CAUSE,NO_SERVICE,[],[],[],[],[],None,90c0d44ddc26d0e649ba4bf712956059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541383,300,0,gtfs-data/rt/2022-05-12T06:58:37/300/0/gtfs_rt...,2022-05-12,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,bb3555a803ce162044eeb695517cf756
541384,300,0,gtfs-data/rt/2022-05-12T06:58:57/300/0/gtfs_rt...,2022-05-12,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,56faa7002e95a6dc399cc9c25e00e8a7
541385,300,0,gtfs-data/rt/2022-05-12T06:59:17/300/0/gtfs_rt...,2022-05-12,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,37ecc903a49bc7c62aaf676608b7c8f9
541386,300,0,gtfs-data/rt/2022-05-12T06:59:37/300/0/gtfs_rt...,2022-05-12,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,6dc452a4367a7318fdf22a7e84b01f4f


## Function for Service Alerts

In [7]:
def get_no_service_alerts(alerts_df,
                          analysis_date: list,
                          itp_id: list,
                          return_df):
    
    alerts_df = alerts_df>>filter(_.calitp_itp_id == itp_id) >> distinct(_.id, _keep_all=True)
    no_service = alerts_df>>filter(_.effect == 'NO_SERVICE')

    #get route_ids where if it appears in no_service
    no_service_rts = [entity[0]['routeId'] for entity in no_service.informed_entities.to_list()]
    
    routes = shared_utils.gtfs_utils.get_route_info(analysis_date, [itp_id])
    routes = routes >> filter(_.route_id.isin(no_service_rts))
    
    trips = shared_utils.gtfs_utils.get_trips(analysis_date, [itp_id])
    trips = trips >> filter(_.route_id.isin(no_service_rts))
    
    if return_df == 'trips':
        return trips
    elif return_df=='routes':
        return routes

In [8]:
no_service = get_no_service_alerts(alert_twodate,'2022-05-12', 300, 'trips')

In [9]:
no_service_agg = no_service>>count(_.route_id)
no_service_agg = no_service_agg.rename(columns={'n':'n_no_service'})

In [10]:
no_service_agg

,route_id,n_no_service
0,3501,104


### Read in existing `rt vs sched` table

In [12]:
rt_sched = utils.read_data()

In [13]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
358,290,SamTrans,1,88,88,2022-06-13,2021-08-15,2022-06-24,1,1,1.00,Monday,June,04 - Oakland


In [14]:
rt_sched_filtered = rt_sched>>filter(_.calitp_itp_id==300, _.service_date == '2022-05-12')

In [15]:
route = rt_sched_filtered>>filter(_.route_id == '3501')
route

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
287,300,Big Blue Bus,0,3501,44,2022-05-12,2022-03-08,2022-09-01,104,0,0.00,Thursday,May,07 - Los Angeles


#### Merge

In [16]:
pd.merge(rt_sched_filtered, no_service_agg, how='left', on= 'route_id')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district,n_no_service
0,300,Big Blue Bus,0,3501,44,2022-05-12,2022-03-08,2022-09-01,104,0,0.00,Thursday,May,07 - Los Angeles,104.00
1,300,Big Blue Bus,0,3488,R10,2022-05-12,2022-03-08,2022-09-01,6,5,0.83,Thursday,May,07 - Los Angeles,NaN
2,300,Big Blue Bus,0,3483,5,2022-05-12,2022-03-08,2022-09-01,24,24,1.00,Thursday,May,07 - Los Angeles,NaN
3,300,Big Blue Bus,0,3491,15,2022-05-12,2022-03-08,2022-09-01,27,27,1.00,Thursday,May,07 - Los Angeles,NaN
4,300,Big Blue Bus,0,3500,43,2022-05-12,2022-03-08,2022-09-01,35,35,1.00,Thursday,May,07 - Los Angeles,NaN
5,300,Big Blue Bus,0,3498,41,2022-05-12,2022-03-08,2022-09-01,39,39,1.00,Thursday,May,07 - Los Angeles,NaN
6,300,Big Blue Bus,0,3495,R3,2022-05-12,2022-03-08,2022-09-01,51,50,0.98,Thursday,May,07 - Los Angeles,NaN
7,300,Big Blue Bus,0,3494,18,2022-05-12,2022-03-08,2022-09-01,53,53,1.00,Thursday,May,07 - Los Angeles,NaN
8,300,Big Blue Bus,0,3492,16,2022-05-12,2022-03-08,2022-09-01,54,54,1.00,Thursday,May,07 - Los Angeles,NaN
9,300,Big Blue Bus,0,3487,9,2022-05-12,2022-03-08,2022-09-01,68,67,0.99,Thursday,May,07 - Los Angeles,NaN


## For two agencies

In [17]:
def get_no_service_alerts2(df,
                          analysis_date: str,
                          itp_ids: list,
                          return_df):
    
    #good for one date as of now. removing calitp_itp_id filter for initial df
    df = df >> distinct(_.id, _keep_all=True)
    no_service = df>>filter(_.effect == 'NO_SERVICE')

    #get route_ids where if it appears in no_service
    no_service_rts = [entity[0]['routeId'] for entity in no_service.informed_entities.to_list()]
    print(no_service_rts)
    #remove None flag if comes up with routes
   # no_service_rts.remove(None)
    
    if return_df == 'trips':
        trips = shared_utils.gtfs_utils.get_trips(analysis_date, itp_ids)
        trips = trips >> filter(_.route_id.isin(no_service_rts))
        
        return trips
    
    elif return_df=='routes':
        routes = shared_utils.gtfs_utils.get_route_info(analysis_date, itp_ids)
        routes = routes >> filter(_.route_id.isin(no_service_rts))
        
        return routes

In [18]:
trip_alerts = get_no_service_alerts2(alert_twodate,
                          analysis_date= '2022-05-11',
                          itp_ids = [290, 300],
                          return_df = 'trips')

['2', None, '17', '3371', '3501']


In [19]:
trip_alerts.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,block_id,calitp_extracted_at,calitp_hash,trip_headsign,direction_id,trip_short_name,calitp_deleted_at,bikes_allowed,wheelchair_accessible,shape_id
512,3742542532326916514,4776306061524233509,171019,17,290,1,73025,2022-05-11,1,2022-05-04,...,None,2022-05-04,qy7fD9dNWqxwcO2Zd8UYQQ==,Miramontes/Moonridge Apts,1,171019,2022-06-07,None,None,170547


In [20]:
def agg_alerts(df):
    no_service = df >> group_by(_.calitp_itp_id) >>count(_.route_id)
    no_service = no_service.rename(columns={'n':'n_no_service'})
    
    return no_service

In [21]:
agg_alerts(trip_alerts)

,calitp_itp_id,route_id,n_no_service
0,290,17,33
1,300,3501,104


In [22]:
def add_to_rtsched_table(trips_alerts_df,
                         rt_sched_table,
                         analysis_date,
                        ):
    no_service = agg_alerts(trips_alerts_df)
    
    rt_sched_table = rt_sched_table>>filter(_.service_date==analysis_date)
    
    #merge trips with no service with rt_vs_sched table
    fulldf = (pd.merge(rt_sched_table, no_service, how='left', on= ['calitp_itp_id','route_id']))
    
    #fill NA with 0 to get the correct number of scheduled trips
    fulldf['n_no_service'] = fulldf['n_no_service'].fillna(value=0)
    fulldf['num_sched_true'] = fulldf['num_sched'] - fulldf['n_no_service']
    
    return fulldf

In [23]:
full_df = add_to_rtsched_table(trip_alerts,
                         rt_sched,
                         analysis_date = '2022-05-11'
                        )

In [24]:
full_df

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district,n_no_service,num_sched_true
0,290,SamTrans,1,141,141,2022-05-11,2021-04-16,2022-07-23,55,55,1.00,Wednesday,May,04 - Oakland,0.00,55.00
1,300,Big Blue Bus,0,3501,44,2022-05-11,2022-03-08,2022-09-01,104,0,0.00,Wednesday,May,07 - Los Angeles,104.00,0.00
2,290,SamTrans,1,80,80,2022-05-11,2021-08-15,2022-06-24,1,1,1.00,Wednesday,May,04 - Oakland,0.00,1.00
3,290,SamTrans,1,88,88,2022-05-11,2021-08-15,2022-06-24,1,1,1.00,Wednesday,May,04 - Oakland,0.00,1.00
4,290,SamTrans,1,14,14,2022-05-11,2021-08-01,2022-06-24,2,2,1.00,Wednesday,May,04 - Oakland,0.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,300,Big Blue Bus,0,3481,3,2022-05-11,2022-03-08,2022-09-01,145,141,0.97,Wednesday,May,07 - Los Angeles,0.00,145.00
82,290,SamTrans,1,ECR,ECR,2022-05-11,2021-04-16,2099-01-01,151,131,0.87,Wednesday,May,04 - Oakland,0.00,151.00
83,300,Big Blue Bus,0,3485,7,2022-05-11,2022-03-08,2022-09-01,153,141,0.92,Wednesday,May,07 - Los Angeles,0.00,153.00
84,300,Big Blue Bus,0,3489,R12,2022-05-11,2022-03-08,2022-09-01,153,152,0.99,Wednesday,May,07 - Los Angeles,0.00,153.00


### written out

In [44]:
# no_service2 = alerts>>filter(_.effect=="NO_SERVICE") >> distinct(_.id, _keep_all=True)

In [45]:
# no_service2

In [46]:
# no_service_rts2 = [entity[0]['routeId'] for entity in no_service2.informed_entities.to_list()]

In [47]:
# no_service_rts2.remove(None)

In [48]:
# no_service_rts2

In [49]:
## have to keep one date instead of 2

In [50]:
# analysis_dates= '2022-05-11'
# itp_ids= [290, 300]

In [51]:
# routes2 = shared_utils.gtfs_utils.get_route_info(analysis_dates, itp_ids)

In [52]:
# routes2.sample()

#### expected output

In [53]:
# routes2 = routes2 >> filter(_.route_id.isin(no_service_rts2))

In [54]:
# routes2

In [55]:
# trips2 = shared_utils.gtfs_utils.get_trips(analysis_dates, itp_ids)
# trips2 = trips2 >> filter(_.route_id.isin(no_service_rts2))

In [56]:
# trips2.sample()

In [57]:
# no_service_agg2 = trips2 >> group_by(_.calitp_itp_id) >>count(_.route_id)

In [58]:
# no_service_agg2 = no_service_agg2.rename(columns={'n':'n_no_service'})

In [59]:
# no_service_agg2

#### Adding rt_sched

In [60]:
##filtering just for service date since we already have the two org
## in the future will need to filter for org
# rt_sched_filtered2 = rt_sched>>filter(_.service_date == '2022-05-12')

In [61]:
# rt_sched_filtered2.sample(5)

In [62]:
# full = (pd.merge(rt_sched_filtered2, no_service_agg2, how='left', on= ['calitp_itp_id','route_id']))

In [63]:
# full.sample(5)

In [64]:
# full>>filter(_.n_no_service.notnull())

In [65]:
# full['n_no_service'] = full['n_no_service'].fillna(value=0)

In [66]:
# full['num_sched_true'] = full['num_sched'] - full['n_no_service']

In [67]:
# full>>arrange(-_.n_no_service)

notes:
    * interesting that SamTran's "No Service" Routes do have vehicle positions.. need to check that out 

### Re-aggregating the `pct_w_vp`

In [68]:
# testing with utils using new true num_sched

utils.get_agg_pct(full_df,
                groupings = ['calitp_itp_id', 'agency_name'],
                sum_sched= 'num_sched_true',
                sum_vp = 'num_vp'
               )

,calitp_itp_id,agency_name,num_sched_true,num_vp,avg
0,290,SamTrans,"1,422.00",1376,0.97
1,300,Big Blue Bus,"1,412.00",1381,0.98


In [69]:
#using regular num_sched
utils.get_agg_pct(full_df,
                groupings = ['calitp_itp_id', 'agency_name'],
                sum_sched= 'num_sched',
                sum_vp = 'num_vp'
               )

,calitp_itp_id,agency_name,num_sched,num_vp,avg
0,290,SamTrans,1455,1376,0.95
1,300,Big Blue Bus,1516,1381,0.91


## Add more agencies
* using May 4th table

In [70]:
may4 = query_sql(f'''
SELECT * FROM `cal-itp-data-infra.views.gtfs_rt_vs_schedule_trips_may4_sample`
''')

In [71]:
may4_orgs = sorted(list(may4.calitp_itp_id.unique()))

In [72]:
#need alerts first
# no_service_all = get_no_service_alerts(alerts,'2022-05-12', 300, 'trips')

In [73]:
def get_alert_data(analysis_date):
    alerts = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
        WHERE date = '{analysis_date}' AND effect = 'NO_SERVICE'
                """)
    ## removing itp for now
       # AND calitp_itp_id in ({itp_list}) 

    return alerts
                

In [74]:
alerts = get_alert_data('2022-05-04')

In [75]:
len(alerts)

175881

In [76]:
alerts.head()

,calitp_itp_id,calitp_url_number,original_file_path,date,id,active_periods,informed_entities,cause,effect,url_translations,header_text_translations,description_text_translations,tts_header_text_translations,tts_description_text_translations,severity_level,key
0,221,0,gtfs-data/rt/2022-05-04T11:00:00/221/0/gtfs_rt...,2022-05-04,c4a44a15-aa31-4bd0-9f9c-eb78c8f4989d,"[{'start': 1647270840, 'end': None}]","[{'agencyId': None, 'routeId': '14212', 'route...",CONSTRUCTION,NO_SERVICE,[],[],[],[],[],None,4d7454408a77e18b7195fa3ced00d28d
1,221,0,gtfs-data/rt/2022-05-04T11:00:20/221/0/gtfs_rt...,2022-05-04,c4a44a15-aa31-4bd0-9f9c-eb78c8f4989d,"[{'start': 1647270840, 'end': None}]","[{'agencyId': None, 'routeId': '14212', 'route...",CONSTRUCTION,NO_SERVICE,[],[],[],[],[],None,cf594c1285e49753669d1931d69c9d45
2,221,0,gtfs-data/rt/2022-05-04T11:00:40/221/0/gtfs_rt...,2022-05-04,c4a44a15-aa31-4bd0-9f9c-eb78c8f4989d,"[{'start': 1647270840, 'end': None}]","[{'agencyId': None, 'routeId': '14212', 'route...",CONSTRUCTION,NO_SERVICE,[],[],[],[],[],None,c03c182780b1b7b0aeba07d09496814c
3,221,0,gtfs-data/rt/2022-05-04T11:01:00/221/0/gtfs_rt...,2022-05-04,c4a44a15-aa31-4bd0-9f9c-eb78c8f4989d,"[{'start': 1647270840, 'end': None}]","[{'agencyId': None, 'routeId': '14212', 'route...",CONSTRUCTION,NO_SERVICE,[],[],[],[],[],None,228f357d376785ae840d00f9ee0bb248
4,221,0,gtfs-data/rt/2022-05-04T11:01:20/221/0/gtfs_rt...,2022-05-04,c4a44a15-aa31-4bd0-9f9c-eb78c8f4989d,"[{'start': 1647270840, 'end': None}]","[{'agencyId': None, 'routeId': '14212', 'route...",CONSTRUCTION,NO_SERVICE,[],[],[],[],[],None,9228a1225407faf23b30e4db7463d98d


In [77]:
len(may4_orgs)

167

In [78]:
len(list(alerts.calitp_itp_id.unique()))

34

In [79]:
# getting error for metrolink, itp_id 323
## will remove from list

may4_orgs.remove(323)

In [80]:
may4_orgs

[4,
 6,
 10,
 11,
 13,
 16,
 17,
 18,
 23,
 24,
 29,
 30,
 34,
 35,
 36,
 37,
 42,
 45,
 48,
 49,
 50,
 56,
 61,
 70,
 71,
 75,
 76,
 77,
 79,
 82,
 83,
 86,
 87,
 95,
 98,
 99,
 101,
 102,
 103,
 105,
 106,
 108,
 110,
 112,
 116,
 118,
 120,
 121,
 122,
 123,
 126,
 127,
 129,
 135,
 137,
 142,
 146,
 148,
 152,
 154,
 159,
 162,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 181,
 182,
 183,
 187,
 188,
 190,
 194,
 198,
 199,
 200,
 201,
 204,
 208,
 210,
 212,
 217,
 218,
 220,
 221,
 226,
 228,
 231,
 232,
 235,
 238,
 239,
 243,
 246,
 247,
 251,
 257,
 259,
 260,
 261,
 264,
 269,
 270,
 271,
 273,
 274,
 278,
 279,
 280,
 281,
 282,
 284,
 287,
 289,
 290,
 293,
 294,
 295,
 296,
 300,
 301,
 305,
 308,
 310,
 312,
 314,
 315,
 320,
 324,
 329,
 331,
 334,
 336,
 337,
 339,
 341,
 343,
 344,
 346,
 349,
 350,
 351,
 356,
 360,
 361,
 366,
 368,
 372,
 374,
 376,
 380,
 381,
 386,
 389,
 473,
 474,
 482,
 483,
 484]

In [81]:
alerts>>filter(_.calitp_itp_id==290)

,calitp_itp_id,calitp_url_number,original_file_path,date,id,active_periods,informed_entities,cause,effect,url_translations,header_text_translations,description_text_translations,tts_header_text_translations,tts_description_text_translations,severity_level,key
52509,290,1,gtfs-data/rt/2022-05-04T01:00:10/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,3de8d398e0ba9fecfeb33d3c1695c852
52510,290,1,gtfs-data/rt/2022-05-04T01:00:30/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,53e03aa743df92b4f35d1621ea3fe667
52511,290,1,gtfs-data/rt/2022-05-04T01:00:50/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,7501a73962dae68731e1253a39be712a
52512,290,1,gtfs-data/rt/2022-05-04T01:01:10/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,6dd6bc2ca85407c5717a660a5c4f5ec9
52513,290,1,gtfs-data/rt/2022-05-04T01:01:30/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,1c5b8e70d7c88e32b5be00ab7ac99e96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142436,290,1,gtfs-data/rt/2022-05-04T00:58:30/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,e2c43b2f6b0cc57c7be12914d1d70a03
142437,290,1,gtfs-data/rt/2022-05-04T00:58:50/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,a0cf36b9ef73d5f5ae06003128926e92
142438,290,1,gtfs-data/rt/2022-05-04T00:59:10/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,6d777da1a0bec378562f07e880b52a1c
142439,290,1,gtfs-data/rt/2022-05-04T00:59:30/290/1/gtfs_rt...,2022-05-04,FS_d961beae-b2bc-4fb7-b46f-abe0ec672ec2,"[{'start': 1651561200, 'end': 1652050800}]","[{'agencyId': 'FS', 'routeId': None, 'routeTyp...",TECHNICAL_PROBLEM,NO_SERVICE,[],[],[],[],[],None,d8c73125e3b1a5a7545b2ffc14d1768a


In [82]:
#starting with a sample of itp_ids

In [83]:
may4_trip_alerts = get_no_service_alerts2(alerts,
                          analysis_date = '2022-05-04',
                          itp_ids = [290],
                          return_df= 'trips')

['14212', None, None, None, '3501', '3371', None, None, '3404', '3397', '3385', '3385', '4940', None, None]


In [84]:
get_no_service_alerts2(alerts,
                          analysis_date = '2022-05-04',
                          itp_ids = [122],
                          return_df= 'trips')

['14212', None, None, None, '3501', '3371', None, None, '3404', '3397', '3385', '3385', '4940', None, None]


,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,block_id,calitp_extracted_at,calitp_hash,trip_headsign,direction_id,trip_short_name,calitp_deleted_at,bikes_allowed,wheelchair_accessible,shape_id


In [85]:
may4_trip_alerts = get_no_service_alerts2(alerts,
                          analysis_date = '2022-05-04',
                          itp_ids = [290],
                          return_df= 'trips')

['14212', None, None, None, '3501', '3371', None, None, '3404', '3397', '3385', '3385', '4940', None, None]


In [87]:
# may4_trip_alerts = get_no_service_alerts2(alerts,
#                           analysis_date = '2022-05-04',
#                           itp_ids = may4_orgs,
#                           return_df= 'trips')

In [88]:
may4_trip_alerts

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,block_id,calitp_extracted_at,calitp_hash,trip_headsign,direction_id,trip_short_name,calitp_deleted_at,bikes_allowed,wheelchair_accessible,shape_id


In [89]:
agg_alerts(may4_trip_alerts)

,calitp_itp_id,route_id,n_no_service


In [90]:
add_to_rtsched_table(may4_trip_alerts,
                         may4,
                         analysis_date ='2022-05-04',
                        )

,agency_name,calitp_url_number,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,calitp_itp_id,route_id,n_no_service,num_sched_true
